In [34]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as pta

In [35]:
# SET PARAMETERS
ticker_symbol = 'MSFT' # can also be 'AAPL GOOG AMZN'
start_date = '2021-01-01'
end_date = '2024-02-24'

df = yf.download(ticker_symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [36]:
# modifying the time range
def get_macd(df, is_daily, is_stock):
    if is_daily:
        factor = 5 if is_stock else 7
        col_prefix = 'weekly'
    else:
        factor = 20 if is_stock else 28         # multiplying weekly factor by 4 - assuming 4 weeks in a month
        col_prefix = 'monthly'
    # Obtain the moving averages for the MACD
    slow_ma = df['Close'].ewm(span=factor*26, adjust=False).mean()
    fast_ma = df['Close'].ewm(span=factor*12, adjust=False).mean()
    # Calculate the MACD and then use that to generate the signal line
    
    df.loc[:, col_prefix + '_macd'] = fast_ma - slow_ma
    df.loc[:, col_prefix + '_signal'] = (
        (fast_ma - slow_ma)
        .ewm(span=factor*9, adjust = False)
        .mean()
    )

    return df.dropna()

In [37]:
df = get_macd(df, is_daily=True, is_stock=True)
df = get_macd(df, is_daily=False, is_stock=True)

In [38]:
# Add columns to calculate differences
df["weekly_diff"] = df["weekly_macd"] - df["weekly_signal"]
df["monthly_diff"] = df["monthly_macd"] - df["monthly_signal"]

In [39]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,weekly_macd,weekly_signal,monthly_macd,monthly_signal,weekly_diff,monthly_diff
Date,,,,,,,,,,,,
2021-01-04,222.529999,223.000000,214.809998,217.690002,211.605301,37130100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-01-05,217.259995,218.520004,215.699997,217.899994,211.809418,23823000,0.003679,0.000160,0.000937,0.000010,0.003519,0.000926
2021-01-06,212.169998,216.490005,211.940002,212.250000,206.317368,35930700,-0.091805,-0.003839,-0.023337,-0.000248,-0.087967,-0.023089
2021-01-07,214.039993,219.339996,213.710007,218.289993,212.188507,27694500,-0.076884,-0.007014,-0.020378,-0.000470,-0.069869,-0.019908
2021-01-08,218.679993,220.580002,217.029999,219.619995,213.481339,22956200,-0.039332,-0.008420,-0.011522,-0.000592,-0.030913,-0.010930


In [40]:
# Add a new column to indicate positive or negative MACD
# Codes:
# 0 when both weekly_diff and monthly_diff are negative.
# 1 when weekly_diff is negative monthly_diff is positive.
# 2 when weekly_diff is positive monthly_diff is negative.
# 3 when both weekly_diff and monthly_diff are positive.

df['codes'] = 0
df.loc[(df['weekly_diff'] < 0) & (df['monthly_diff'] < 0), 'codes'] = 0
df.loc[(df['weekly_diff'] < 0) & (df['monthly_diff'] > 0), 'codes'] = 1
df.loc[(df['weekly_diff'] > 0) & (df['monthly_diff'] < 0), 'codes'] = 2
df.loc[(df['weekly_diff'] > 0) & (df['monthly_diff'] > 0), 'codes'] = 3
df

,Open,High,Low,Close,Adj Close,Volume,weekly_macd,weekly_signal,monthly_macd,monthly_signal,weekly_diff,monthly_diff,codes
Date,,,,,,,,,,,,,
2021-01-04,222.529999,223.000000,214.809998,217.690002,211.605301,37130100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2021-01-05,217.259995,218.520004,215.699997,217.899994,211.809418,23823000,0.003679,0.000160,0.000937,0.000010,0.003519,0.000926,3
2021-01-06,212.169998,216.490005,211.940002,212.250000,206.317368,35930700,-0.091805,-0.003839,-0.023337,-0.000248,-0.087967,-0.023089,0
2021-01-07,214.039993,219.339996,213.710007,218.289993,212.188507,27694500,-0.076884,-0.007014,-0.020378,-0.000470,-0.069869,-0.019908,0
2021-01-08,218.679993,220.580002,217.029999,219.619995,213.481339,22956200,-0.039332,-0.008420,-0.011522,-0.000592,-0.030913,-0.010930,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-16,407.959991,408.290009,403.440002,404.059998,404.059998,22281100,21.779783,18.530474,30.654584,18.935743,3.249309,11.718840,3
2024-02-20,403.239990,404.489990,398.010010,402.790009,402.790009,24307900,21.716925,18.669016,30.806155,19.066908,3.047909,11.739247,3
2024-02-21,400.170013,402.290009,397.220001,402.179993,402.179993,18631100,21.635498,18.797993,30.952189,19.198237,2.837505,11.753952,3


In [41]:
df['switch_code'] = 0  # Initialize the new switch column with zeros

df.loc[(df['codes'].shift() != 1) & (df['codes'] == 1), 'switch_code'] = 1  # down trend start (only weekly neg)

df.loc[(df['codes'].shift() != 0) & (df['codes'] == 0), 'switch_code'] = 2  # down trend (both neg)

df.loc[(df['codes'].shift() != 2) & (df['codes'] == 2), 'switch_code'] = 3  # up trend (only weekly pos)

df.loc[(df['codes'].shift() != 3) & (df['codes'] == 3), 'switch_code'] = 4  # up trend (both pos)

df


,Open,High,Low,Close,Adj Close,Volume,weekly_macd,weekly_signal,monthly_macd,monthly_signal,weekly_diff,monthly_diff,codes,switch_code
Date,,,,,,,,,,,,,,
2021-01-04,222.529999,223.000000,214.809998,217.690002,211.605301,37130100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2
2021-01-05,217.259995,218.520004,215.699997,217.899994,211.809418,23823000,0.003679,0.000160,0.000937,0.000010,0.003519,0.000926,3,4
2021-01-06,212.169998,216.490005,211.940002,212.250000,206.317368,35930700,-0.091805,-0.003839,-0.023337,-0.000248,-0.087967,-0.023089,0,2
2021-01-07,214.039993,219.339996,213.710007,218.289993,212.188507,27694500,-0.076884,-0.007014,-0.020378,-0.000470,-0.069869,-0.019908,0,0
2021-01-08,218.679993,220.580002,217.029999,219.619995,213.481339,22956200,-0.039332,-0.008420,-0.011522,-0.000592,-0.030913,-0.010930,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-16,407.959991,408.290009,403.440002,404.059998,404.059998,22281100,21.779783,18.530474,30.654584,18.935743,3.249309,11.718840,3,0
2024-02-20,403.239990,404.489990,398.010010,402.790009,402.790009,24307900,21.716925,18.669016,30.806155,19.066908,3.047909,11.739247,3,0
2024-02-21,400.170013,402.290009,397.220001,402.179993,402.179993,18631100,21.635498,18.797993,30.952189,19.198237,2.837505,11.753952,3,0


In [45]:
df['atr'] = pta.atr(df['High'], df['Low'], df['Close'], window=14, fillna=False)
df['atr'].replace(False, float('nan'), inplace=True)
df

/var/folders/bc/440d68zj0qgfv8fdlqk34yyr0000gn/T/ipykernel_72978/1414189375.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['atr'] = pta.atr(df['High'], df['Low'], df['Close'], window=14, fillna=False)


,Open,High,Low,Close,Adj Close,Volume,weekly_macd,weekly_signal,monthly_macd,monthly_signal,weekly_diff,monthly_diff,codes,switch_code,atr
Date,,,,,,,,,,,,,,,
2021-01-04,222.529999,223.000000,214.809998,217.690002,211.605301,37130100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,2,NaN
2021-01-05,217.259995,218.520004,215.699997,217.899994,211.809418,23823000,0.003679,0.000160,0.000937,0.000010,0.003519,0.000926,3,4,NaN
2021-01-06,212.169998,216.490005,211.940002,212.250000,206.317368,35930700,-0.091805,-0.003839,-0.023337,-0.000248,-0.087967,-0.023089,0,2,NaN
2021-01-07,214.039993,219.339996,213.710007,218.289993,212.188507,27694500,-0.076884,-0.007014,-0.020378,-0.000470,-0.069869,-0.019908,0,0,NaN
2021-01-08,218.679993,220.580002,217.029999,219.619995,213.481339,22956200,-0.039332,-0.008420,-0.011522,-0.000592,-0.030913,-0.010930,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-16,407.959991,408.290009,403.440002,404.059998,404.059998,22281100,21.779783,18.530474,30.654584,18.935743,3.249309,11.718840,3,0,6.682610
2024-02-20,403.239990,404.489990,398.010010,402.790009,402.790009,24307900,21.716925,18.669016,30.806155,19.066908,3.047909,11.739247,3,0,6.668137
2024-02-21,400.170013,402.290009,397.220001,402.179993,402.179993,18631100,21.635498,18.797993,30.952189,19.198237,2.837505,11.753952,3,0,6.589699


In [46]:
df.to_csv('output.csv')

In [17]:
def backtest(Open, Close, Low, weekly_opp, daily_opp):
    '''
    Perform the buying/selling. The strategy is as follows:
    1. Buy if the weekly MACD > weekly signal and daily MACD > daily signal
    2. Wait until the daily signal > daily MACD, set the SL to the LOD
    3. Update the stop loss if the MACD crosses below the signal again
    Parameters
    Open : np.array
    The open prices over each day
    Close: np.array,
    The close prices for each day
    Low : np.array
    The low prices over each day weekly_opp: np.array
    An array to show when the weekly MACD is above the weekly signal daily_opp: np.array
    An array to show when the daily MACD is above the daily signal
    Returns
    bought, sold, percs: np.array
    The outcome from the backtest
    '''
    # Empty list to store the simulation outcome
    percs = []
    # Flags for the backtest. Holding is used to flag whether we are currently
    # holding an asset. The sl_set is used to determine if we the
    # stop loss has been fixed and we waiting for it to be hit.
    holding = False
    sl_set = False
    for idx in range(1, Open.shape[0]):
        print(holding)
        # Check the previous day, and see if a buy signal was met. If so, enter on the open
        # and we're in the trade for at least the first day
        if weekly_opp[idx-1] and daily_opp[idx-1] and not holding:
            percs. append(Close[idx]/Open[idx]-1)
            holding = True
            continue
        elif holding:
            # In this case, the daily MACD < daily signal, and we set the SL
            # to the day's low price
            if not sl_set and not daily_opp[idx]:
                sl_set = True
                sl_val = Low[idx]
                percs. append(Close[idx]/Close[idx-1]-1)
                continue
            # If the SL was not hit, and the MACD crosses again, we update the SL
            elif (
            sl_set
            and Low[idx] > sl_val 
            and daily_opp[idx-1]
            and not daily_opp[idx]
            ):
                sl_val = Low[idx]
                percs.append(Close[idx]/Close[idx-1]-1)
                continue
            # In this case, the stop loss is met, and we exit the trade
            elif sl_set and Low[idx] < sl_val:
                holding = False
                sl_set = False
                percs.append(sl_val/Close[idx-1]-1)
                del sl_val
                continue
            else:
                percs. append(Close[idx]/Close[idx-1]-1)
        else:
            percs. append (0)
    return np.array(percs)

In [18]:
# df.loc[1:, 'percs'] = backtest(
#     df['Open'].values,
#     df['Close'].values,
#     df['Low'].values,
# (df['weekly_macd'] > df['weekly_signal']).values,
# (df['daily_macd'] > df['daily_signal']). values,
# )

percs = backtest(
    df['Open'].values,
    df['Close'].values,
    df['Low'].values,
    (df['weekly_macd'] > df['weekly_signal']).values,
    (df['daily_macd'] > df['daily_signal']).values
)

# Assign the values to the 'percs' column starting from row 2
df['percs'] = np.nan  # Initialize the column with NaN
df['percs'][1:] = percs  # Assign values starting from row 2

df['equity_strat'] = np.cumprod(1+df['percs'])
df['equity_bh'] = np. cumprod(1+df['Close'].pct_change())

KeyError: 'daily_macd'

In [ ]:
df.head(20)

,Open,High,Low,Close,Adj Close,Volume,daily_macd,daily_signal,weekly_macd,weekly_signal,percs,equity_strat,equity_bh
Date,,,,,,,,,,,,,
2021-01-04,87.876999,88.032501,85.392502,86.412003,86.412003,38038000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
2021-01-05,86.250000,87.383499,85.900749,87.045998,87.045998,22906000,0.050575,0.010115,0.011107,0.000483,0.000000,1.000000,1.007337
2021-01-06,85.131500,87.400002,84.949997,86.764503,86.764503,52042000,0.067168,0.021526,0.016749,0.001190,0.019182,1.019182,1.004079
2021-01-07,87.002998,89.419998,86.852501,89.362503,89.362503,45300000,0.286650,0.074550,0.067631,0.004079,0.029943,1.049700,1.034145
2021-01-08,89.399002,90.491997,88.676750,90.360497,90.360497,41012000,0.534954,0.166631,0.133543,0.009708,0.011168,1.061423,1.045694
2021-01-11,89.303497,89.715752,88.026001,88.335999,88.335999,24194000,0.561900,0.245685,0.160786,0.016276,-0.022405,1.037642,1.022265
2021-01-12,87.695999,88.902000,86.265503,87.327499,87.327499,27140000,0.496158,0.295780,0.168984,0.022916,-0.011417,1.025795,1.010595
2021-01-13,86.929001,88.251747,86.900497,87.720001,87.720001,21882000,0.470307,0.330685,0.183585,0.029901,0.004495,1.030406,1.015137
2021-01-14,87.681000,88.750504,86.669998,87.009003,87.009003,23590000,0.387976,0.342143,0.184942,0.036642,-0.008105,1.022054,1.006909
